In [76]:
import polars as pl
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq

In [77]:
def convert_tsv_to_parquet(tsv_file_path, parquet_dir):
    """
    Converts a TSV file to a Parquet file
    """

    parquet_file_name = os.path.basename(tsv_file_path).replace('.csv', '.parquet')
    parquet_file_path = os.path.join(parquet_dir, parquet_file_name)

    df = pl.read_csv(tsv_file_path, separator=',', has_header=True, ignore_errors=True)
    df.write_parquet(parquet_file_path)

    return parquet_file_path

In [78]:
path = convert_tsv_to_parquet("../pipeline/data/raw_data/tsv/archive/rotten_tomatoes_movies.csv", "../pipeline/data/raw_data/parquet/")

In [79]:
df = pl.read_parquet(path).drop_nulls()

In [80]:
df.columns

['id',
 'title',
 'audienceScore',
 'tomatoMeter',
 'rating',
 'ratingContents',
 'releaseDateTheaters',
 'releaseDateStreaming',
 'runtimeMinutes',
 'genre',
 'originalLanguage',
 'director',
 'writer',
 'boxOffice',
 'distributor',
 'soundMix']

In [81]:
df.head()

id,title,audienceScore,tomatoMeter,rating,ratingContents,releaseDateTheaters,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director,writer,boxOffice,distributor,soundMix
str,str,i64,i64,str,str,str,str,i64,str,str,str,str,str,str,str
"""monsoon_weddin…","""Monsoon Weddin…",87,95,"""R""","""['Some Sex Rel…","""2002-02-22""","""2002-09-24""",100,"""Comedy, Drama""","""English""","""Mira Nair""","""Sabrina Dhawan…","""$13.9M""","""USA Films""","""Dolby Stereo, …"
"""robin_hood_201…","""Robin Hood""",40,15,"""PG-13""","""['Extended Seq…","""2018-11-21""","""2019-02-19""",118,"""Action, Advent…","""English""","""Otto Bathurst""","""Ben Chandler,D…","""$30.8M""","""Lionsgate Film…","""Dolby Atmos"""
"""my_giant""","""My Giant""",25,21,"""PG""","""['Violence', '…","""1998-04-10""","""1998-09-29""",103,"""Comedy, Drama""","""English""","""Michael Lehman…","""Billy Crystal,…","""$8.0M""","""Columbia Pictu…","""SDDS, Dolby SR…"
"""west_side_stor…","""West Side Stor…",93,91,"""PG-13""","""['Some Strong …","""2021-12-10""","""2022-03-02""",156,"""Musical, Roman…","""English""","""Steven Spielbe…","""Tony Kushner""","""$38.5M""","""20th Century S…","""Dolby Digital,…"
"""youth_2015""","""Youth""",69,71,"""R""","""['Language', '…","""2015-12-04""","""2016-03-01""",118,"""Drama""","""English""","""Paolo Sorrenti…","""Paolo Sorrenti…","""$2.7M""","""Fox Searchligh…","""Dolby Atmos"""


In [82]:
imdb = pq.read_table("../pipeline/data/raw_data/parquet/title.basics.parquet")

ComputeError: invalid utf-8 sequence

In [ ]:
imdb.head()